In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import Session, joinedload
from itemchm.utils import overall_rate

import pickle as pk
import itemchm.persistence.entities as db
from itemchm.hybrid_recommender import HybridRecommender
from itemchm.entities_repr import User, Book


In [2]:
# Create engine
ENGINE = create_engine("sqlite:///../../bookshelf.db", echo=True)

In [3]:
with Session(ENGINE) as session:
    db_users = session.query(db.User).all()
    db_books = session.query(db.Book).options(joinedload(db.Book.genres)).all()
    db_user_books = session.query(db.UserBook).all()

2024-08-28 17:06:46,524 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-28 17:06:46,526 INFO sqlalchemy.engine.Engine SELECT "Users".id AS "Users_id", "Users".name AS "Users_name", "Users".email AS "Users_email", "Users".features AS "Users_features" 
FROM "Users"
2024-08-28 17:06:46,527 INFO sqlalchemy.engine.Engine [generated in 0.00089s] ()
2024-08-28 17:06:46,535 INFO sqlalchemy.engine.Engine SELECT "Books".id AS "Books_id", "Books".name AS "Books_name", "Books".author AS "Books_author", "Books".language AS "Books_language", "Books".year AS "Books_year", "BookGenre_1"."booksId" AS "BookGenre_1_booksId", "BookGenre_1"."genresName" AS "BookGenre_1_genresName" 
FROM "Books" LEFT OUTER JOIN "BookGenre" AS "BookGenre_1" ON "Books".id = "BookGenre_1"."booksId"
2024-08-28 17:06:46,536 INFO sqlalchemy.engine.Engine [generated in 0.00078s] ()
2024-08-28 17:06:47,092 INFO sqlalchemy.engine.Engine SELECT "UserBooks".id AS "UserBooks_id", "UserBooks"."readRatio" AS "UserBooks_readRatio",

In [4]:
users_dict : dict[int, User] = {}

for db_user in db_users:
    user = User(db_user.id, db_user.name, {})
    users_dict[db_user.id] = user

books : list[Book] = []

for db_book in db_books:
    genres = [book_genre.genresName for book_genre in db_book.genres]
    book = Book(db_book.id, db_book.name, db_book.author, db_book.year, db_book.language, genres)
    books.append(book)

In [5]:
for user_book in db_user_books:
    rating = overall_rate(user_book)
    user = users_dict[user_book.userId]
    user.ratings[user_book.bookId] = rating

# Required Input
users = list(users_dict.values())

In [9]:
with open("books.pkl", "wb") as f:
    books_pickle = pk.dump(books, f)
with open("users.pkl", "wb") as u:
    users_pickle = pk.dump(users, u)

with open("books.pkl", "rb") as f:
    books_pk = pk.load(f)

with open("users.pkl", "rb") as u:
    users_pk = pk.load(u)


In [11]:
with open("books.pkl", "rb") as f:
    books_pk = pk.load(f)

with open("users.pkl", "rb") as u:
    users_pk = pk.load(u)

In [13]:
system = HybridRecommender(books_pk, users_pk)
system.compute_data()

Start Mixing Matrix
Start building Item Rating Similitud
Start building Item Rating Matrix


KeyboardInterrupt: 

In [10]:
with open("system.pk", "wb") as s:
    sy = pk.dump(system, s)